In [3]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Initialize the SQLite database
conn = sqlite3.connect("lms.db")
cursor = conn.cursor()

# Create the tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS courses (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        description TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS progress (
        user_id INTEGER,
        course_id INTEGER,
        completed BOOLEAN,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (course_id) REFERENCES courses(id)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS teachers (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS marks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        student_id INTEGER,
        course_id INTEGER,
        mark INTEGER,
        FOREIGN KEY (student_id) REFERENCES users(id),
        FOREIGN KEY (course_id) REFERENCES courses(id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS announcements (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        teacher_id INTEGER,
        message TEXT,
        FOREIGN KEY (teacher_id) REFERENCES teachers(id),
        FOREIGN KEY (course_id) REFERENCES courses(id)
    )
''')



# Create the tkinter window
root = tk.Tk()
root.title("Learning Management System")

root.configure(bg="#5D6D7E")

width= root.winfo_screenwidth() 
height= root.winfo_screenheight()
#setting tkinter window size
root.geometry("%dx%d" % (width, height))


# Function to show available courses
def show_courses():
    main_menu_frame.pack_forget()
    main_menu_frame.pack()
    courses_list.delete(0, tk.END)
    for row in cursor.execute("SELECT title FROM courses"):
        courses_list.insert(tk.END, row[0])

# Function to enroll in a course
def enroll_course():
    selected_course = courses_list.get(courses_list.curselection())
    course_id = cursor.execute("SELECT id FROM courses WHERE title=?", (selected_course,)).fetchone()[0]
    cursor.execute("INSERT INTO progress (user_id, course_id, completed) VALUES (?, ?, ?)", (current_user_id, course_id, False))
    conn.commit()
    show_progress()  # Refresh the list of courses after enrolling

# Function to show enrolled courses and track progress
def show_progress():
     
    progress_list.delete(0, tk.END)
    for row in cursor.execute("SELECT courses.title, progress.completed FROM courses JOIN progress ON courses.id = progress.course_id WHERE progress.user_id=?", (current_user_id,)):
        progress_list.insert(tk.END, f"Course: {row[0]}, Completed: {'Yes' if row[1] else 'No'}")
      

# Function to mark a course as completed
def mark_completed():
    selected_progress = progress_list2.get(progress_list2.curselection())
    course_title = selected_progress.split("Course: ")[1].strip()
    
    # Query the database for the course_id
    cursor.execute("SELECT id FROM courses WHERE title=?", (course_title,))
    course_id = cursor.fetchone()
    
    if course_id:
        course_id = course_id[0]
        cursor.execute("UPDATE progress SET completed = ? WHERE user_id = ? AND course_id = ?", (True, crnt_user, course_id))
        conn.commit()
        show_progress()
    else:
        print("Course not found in the database.")

# Function to populate the listbox with usernames and IDs from the 'users' table for teacher frame
def show_users():
    users_listbox.delete(0, tk.END)
    for row in cursor.execute("SELECT id, username FROM users"):
        users_listbox.insert(tk.END, f"ID: {row[0]}, Username: {row[1]}")
    
# Function for user login
def login():
    username = username_entry.get()
    password = password_entry.get()

    # Check if the user is a regular user
    user_data = cursor.execute("SELECT id, username, password FROM users WHERE username=? AND password=?", (username, password)).fetchone()

    if user_data:
        # User login
        global current_user_id
        current_user_id = user_data[0]
        login_frame.pack_forget()
        main_menu_frame.pack()
        show_courses()
        show_progress()
    else:
        # Check if the user is a teacher
        teacher_data = cursor.execute("SELECT id, username, password FROM teachers WHERE username=? AND password=?", (username, password)).fetchone()

        if teacher_data:
            flag=True
            current_user_id=teacher_data[0]
            login_frame.pack_forget()
            teacher_frame.pack()
            show_users()
        else:
            prompt=messagebox.showinfo("Login Error","Invalid Credential")
            
#teacher function
def add_mark():
    student_id = crnt_user
    select=progress_list2.get(progress_list2.curselection())
    title=select.split("Course: ")[1].strip()
    print(title)
    course_id = cursor.execute("""SELECT id from courses WHERE title=?""",(title,)).fetchone()[0]
    mark = int(mark_entry.get())
    cursor.execute("INSERT INTO marks (student_id, course_id, mark) VALUES (?, ?, ?)", (student_id, course_id, mark))
    conn.commit()
    mark_entry.delete(0, tk.END)

def add_announcement():
    message = announcement_entry.get()
    teacher_id = current_user_id  # Assuming the teacher's ID is stored in current_user_id
    cursor.execute("INSERT INTO announcements (teacher_id, message) VALUES (?, ?)", (teacher_id, message))
    conn.commit()
    announcement_entry.delete(0, tk.END)

def add_course():
    title = course_title.get()
    description = course_description.get()
    cursor.execute("INSERT INTO courses (title, description) VALUES (?, ?)", (title, description))
    conn.commit()
    course_title.delete(0, tk.END)
    course_description.delete(0, tk.END)
    show_courses()

def show_progress2():
    selected=users_listbox.get(users_listbox.curselection())
    global crnt_user
    crnt_user=int(selected.split("ID: ")[1].split(", Username:")[0].strip())
    progress_list2.delete(0, tk.END)
    for row in cursor.execute("SELECT courses.title, progress.completed FROM courses JOIN progress ON courses.id = progress.course_id WHERE progress.user_id=?", (crnt_user,)):
        progress_list2.insert(tk.END, f"Course: {row[0]}")

# Function to switch to the sign-up frame
def switch_to_login_frame():
    signup_frame.pack_forget()
    login_frame.pack()
def switch_to_signup_frame():
    signup_frame.pack()
    login_frame.pack_forget()

def switch_to_teacher_frame():
    main_menu_frame.pack_forget()
    teacher_frame.pack()

def show_marks():
    selected=progress_list.get(progress_list.curselection())
    course=selected.split("Course: ")[1].split(", Completed:")[0].strip()
    course_id=cursor.execute("""SELECT id FROM courses WHERE title=?""",(course,)).fetchone()[0]
    course_complete=cursor.execute("""SELECT completed FROM progress  WHERE user_id=? AND course_id=?""",(current_user_id,course_id)).fetchone()[0]
    if course_complete:
        marks=str(cursor.execute("""SELECT mark From marks WHERE student_id=? AND course_id=?""",(current_user_id,course_id)).fetchone()[0])
        print(marks,course_id)
        mark_lbl=tk.Label(main_menu_frame,text="Marks Obatined are "+marks+" off 100",font=("SImSun",14))
        mark_lbl.pack()
    else:
        messagebox.showinfo("Course Incomplete","The course is not yet complete to show marks.")
        username_entry.delete(0, tk.END)
        password_entry.delete(0, tk.END)

def show_announcements():
    announcement=cursor.execute("""SELECT message FROM announcements""")
    for msg in announcement:
        announcements_txt.insert(tk.END,msg)
# Function to create a new user
def create_account():
    flag=True
    username = signup_username_entry.get()
    password = signup_password_entry.get()
    cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    signup_username_entry.delete(0, tk.END)
    signup_password_entry.delete(0, tk.END)
    switch_to_login_frame()

def create_account_teacher():
    flag=True
    username = signup_username_entry.get()
    password = signup_password_entry.get()
    cursor.execute("INSERT INTO teachers (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    signup_username_entry.delete(0, tk.END)
    signup_password_entry.delete(0, tk.END)
    switch_to_login_frame()

# Create and configure frames
login_frame = tk.Frame(root, bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2, height=500)
login_frame.pack(pady=150)

signup_frame = tk.Frame(root, bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2, height=500)
signup_frame.pack_forget()

main_menu_frame = tk.Frame(root,bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2)

teacher_frame=tk.Frame(root,bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2)

marks_frame=tk.Frame(root,bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2)
# Create and configure widgets
login_label = tk.Label(login_frame, text="Login", font=("Times", "50", "bold italic"), bg="#2E4053")
username_label = tk.Label(login_frame, text="Username:", font=("Times", "20"), bg="#2E4053")
password_label = tk.Label(login_frame, text="Password:", font=("Times", "20"), bg="#2E4053")
username_entry = tk.Entry(login_frame, width=40)
password_entry = tk.Entry(login_frame, show="*", width=40)
login_button = tk.Button(login_frame, text="Login", command=login, bg="#DAF7A6")
create_account_button = tk.Button(login_frame, text="Create Account", command=switch_to_signup_frame, bg="#DAF7A6")

signup_label = tk.Label(signup_frame, text="Create Account", font=("Times", "50", "bold italic"), bg="#2E4053")
signup_username_label = tk.Label(signup_frame, text="Username:", font=("Times", "20"), bg="#2E4053")
signup_password_label = tk.Label(signup_frame, text="Password:", font=("Times", "20"), bg="#2E4053")
signup_username_entry = tk.Entry(signup_frame, width=40)
signup_password_entry = tk.Entry(signup_frame, show="*", width=40)
signup_button = tk.Button(signup_frame, text="Create Account as a student", command=create_account, bg="#DAF7A6")
signup_button2 = tk.Button(signup_frame, text="Create Account as a Teacher", command=create_account_teacher, bg="#DAF7A6")


mark_label = tk.Label(teacher_frame, text="Add Marks", font=('Times', 14), bg="#2E4053")
users_listbox = tk.Listbox(teacher_frame, selectmode=tk.SINGLE, width=50)
progress_list2 = tk.Listbox(teacher_frame, selectmode=tk.SINGLE, width=50)
show_enrolled_courses=tk.Button(teacher_frame,text="Show Enrolled Courses",command=show_progress2,bg="#DAF7A6")
mark_completed_button = tk.Button(teacher_frame, text="Mark as Completed", command=mark_completed, bg="#DAF7A6")
mark_entry = tk.Entry(teacher_frame, width=30)
add_mark_button = tk.Button(teacher_frame, text="Add Mark", command=add_mark, bg="#DAF7A6")
# Create a Tkinter listbox
announcement_label = tk.Label(teacher_frame, text="Add Announcement", font=('Times', 14), bg="#2E4053")
announcement_entry = tk.Entry(teacher_frame, width=30)
add_announcement_button = tk.Button(teacher_frame, text="Add Aouncement", command=add_announcement, bg="#DAF7A6")
course_label = tk.Label(teacher_frame, text="Add a New Course",font=('Times',14), bg="#2E4053")
course_title = tk.Entry(teacher_frame, width=30)
course_description = tk.Entry(teacher_frame, width=30)
add_course_button = tk.Button(teacher_frame, text="Add Course", command=add_course, bg="#DAF7A6")

wel=tk.Label(main_menu_frame,text="Welcome to Learning Management System",font=("Times", 20, "bold italic"), bg="#2E4053")
lbl=tk.Label(main_menu_frame,text="Available Courses",font=("Times",14),fg="green",bg="#2E4053")
courses_list = tk.Listbox(main_menu_frame, selectmode=tk.SINGLE, width=50)
enroll_button = tk.Button(main_menu_frame, text="Enroll in Course", command=enroll_course, bg="#DAF7A6")
progress_label = tk.Label(main_menu_frame, text="Course Progress",font=('Times',14), bg="#2E4053")
progress_list = tk.Listbox(main_menu_frame, selectmode=tk.SINGLE, width=50)
show_marks_btn=tk.Button(main_menu_frame,text="Show Marks",command=show_marks,bg="#DAF7A6")
announcements_txt=tk.Text(main_menu_frame,height=5,width=50)
show_announcement_btn=tk.Button(main_menu_frame,text="Show Announcements",bg="#DAF7A6",command=show_announcements)
# Place widgets on the frames
#login frame
login_label.pack()
username_label.pack()
username_entry.pack()
password_label.pack()
password_entry.pack()
login_button.pack()
create_account_button.pack()

#signup frame
signup_label.pack()
signup_username_label.pack()
signup_username_entry.pack()
signup_password_label.pack()
signup_password_entry.pack()
signup_button.pack()
signup_button2.pack()


#main menu frame
wel.pack()
lbl.pack()
course_label.pack()
course_title.pack()
course_description.pack()
courses_list.pack()
enroll_button.pack()
progress_label.pack()
progress_list.pack()
show_marks_btn.pack()
announcements_txt.pack()
show_announcement_btn.pack()

#teacher frame
add_course_button.pack()
mark_label.pack()
users_listbox.pack()
progress_list2.pack()
show_enrolled_courses.pack()
mark_entry.pack()
mark_completed_button.pack()
add_mark_button.pack()
announcement_label.pack()
announcement_entry.pack()
add_announcement_button.pack()


main_menu_frame.pack_forget()
# Start the tkinter main loop
root.mainloop()

# Close the database connection
conn.close()